In [90]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import datetime
import time
import speech_recognition as sr
import pandas as pd

In [91]:
test_sentiment_df = pd.read_csv('CoreData.csv', sep=';')

In [92]:
replies_text = test_sentiment_df[pd.to_numeric(test_sentiment_df['Reply'], errors='coerce').isnull()]

In [93]:
replies_text = replies_text['Reply']

In [94]:
replies_text = replies_text.dropna()

In [95]:
len(replies_text)

10619

In [96]:
# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']



In [98]:
thought = replies_text.iloc[1600]

encoded_thought = tokenizer(thought, return_tensors='pt')
print(thought)


output = model(**encoded_thought)

scores = output[0][0].detach().numpy()
scores = softmax(scores)

for i in range(len(scores)):
    
    l = labels[i]
    s = scores[i]
    print(l,s)

shame
Negative 0.845995
Neutral 0.13474905
Positive 0.019255958


In [ ]:
# thought_entries_df = pd.DataFrame(columns = ['Situation','Automatic Thoughts','Emotions/Feelings','Distress','Sent_Negatve','Sent_Neutral','Sent_Positive','Timestamp'])

In [99]:
def thought_entry():

    situation = input('Briefly describe the situation')
    thought = input('What is your thought?')
    emotion = input('How does it make you feel?')
    distress = input('Can you input a level of distress between 1 to 10?')

    timestamp = datetime.datetime.now()

    encoded_thought = tokenizer(thought, return_tensors='pt')

    output = model(**encoded_thought)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    for i in range(len(scores)):
    
        l = labels[i]
        s = scores[i]

    thought_entries_df.loc[len(thought_entries_df.index)] = [situation, thought, emotion, distress, scores[0],scores[1],scores[2],timestamp.strftime("%B %d, %Y, %H:%Mh")]
    




In [ ]:
""" def thought_entry_TEST(thought):

    situation = 'situation'
    emotion = 'emotion'
    distress = 'distress'

    encoded_thought = tokenizer(thought, return_tensors='pt')

    output = model(**encoded_thought)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    for i in range(len(scores)):
    
        l = labels[i]
        s = scores[i]

    thought_entries_df.loc[len(thought_entries_df.index)] = [situation, thought, emotion, distress, scores[0],scores[1],scores[2]]"""

" def thought_entry_TEST(thought):\n\n    situation = 'situation'\n    emotion = 'emotion'\n    distress = 'distress'\n\n    encoded_thought = tokenizer(thought, return_tensors='pt')\n\n    output = model(**encoded_thought)\n\n    scores = output[0][0].detach().numpy()\n    scores = softmax(scores)\n\n    for i in range(len(scores)):\n    \n        l = labels[i]\n        s = scores[i]\n\n    thought_entries_df.loc[len(thought_entries_df.index)] = [situation, thought, emotion, distress, scores[0],scores[1],scores[2]]"

In [ ]:
""" for reply in replies_text['Reply'].iloc[:1000]:
    thought_entry_TEST(reply) """

" for reply in replies_text['Reply'].iloc[:1000]:\n    thought_entry_TEST(reply) "

In [ ]:
""" thought_entries_df.to_csv('test_count_frequency.csv', index = False) """

" thought_entries_df.to_csv('test_count_frequency.csv', index = False) "

In [ ]:
# basic sentiment analysis; explore other models ahead

# learn more about CBT to prepare the inputs of the user

In [ ]:
# User inputs

# Situation -> Brief Description of the situation
# Automatic Thoughts -> The thought or general overview of the thought going through your head
# Emotions/Feelings -> Just the emotions/feelings [perhaps make this weigh more on the sentiment score depending on if the emotion is positive
# or negative]


# Psychologist Inputs

# Hot Thought: Finding a way to lump many thoughts of a particular user and trying to identify the trend 
# To go together with the user once the hot thought has been determined: Evidence that supports the hot thought vs. Evidence against it
# Propose alternative Thoughts to the Hot Thought

In [ ]:
# thought_entry()

In [ ]:

## depending on the distress level (if it's very high or very low, make it weigh on the sentiment)
## also make bins for the sentiments (very positive, positive, neutral, negative, very negative)

In [100]:
# need to find out how to create an dataframe for each user

thought_entries_df = pd.DataFrame(columns = ['Situation','Automatic Thoughts','Emotions/Feelings','Distress',
                                                'Sentiment','Timestamp'])

In [101]:
# break this into different functions

def thought_entry_voice():
    
    
    # speech recognition objects

    r = sr.Recognizer()
    mic = sr.Microphone()   

    # series of inputs 

    situation = input('Briefly describe the situation')

    # input for the thought; find a better way to do this
   
    with mic as source:
        r.adjust_for_ambient_noise(source, 1)
        print('What is your thought?')
        audio = r.listen(source)
        time.sleep(1)
        
    thought = r.recognize_google(audio)
    emotion = input('How does it make you feel?')
    distress = input('Can you input a level of distress between 1 to 10?')

    # create the timestamp of the entry

    timestamp = datetime.datetime.now()

    # tokenizing

    encoded_thought = tokenizer(thought, return_tensors='pt')

    output = model(**encoded_thought)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    for i in range(len(scores)):
    
        l = labels[i]
        s = scores[i]


    scores_list = [scores[i] for i in range(len(scores))]
    labels_list = [labels[i] for i in range(len(scores))]
    zipped = list(zip(scores_list,labels_list))

    thought_entries_df.loc[len(thought_entries_df.index)] = [situation, thought, emotion, distress, max(zipped)[1],timestamp.strftime("%B %d, %Y, %H:%Mh")]
    
    return thought_entries_df




In [102]:
def thought_test (thought):

    # thought = input('What is your thought?')

    timestamp = datetime.datetime.now()

    encoded_thought = tokenizer(thought, return_tensors='pt')

    output = model(**encoded_thought)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    for i in range(len(scores)):
    
        l = labels[i]
        s = scores[i]

    
    scores_list = [scores[i] for i in range(len(scores))]
    labels_list = [labels[i] for i in range(len(scores))]
    zipped = list(zip(scores_list,labels_list))


    return max(zipped)[1]

In [103]:
thought_test('I am very happy today')

'Positive'

In [130]:
thought_entries_df = pd.DataFrame(columns = ['Thoughts','Sent_Negative','Sent_Neutral','Sent_Positive',
                                                'Sentiment'])

In [131]:
def thought_entry_specialist(thought):



    timestamp = datetime.datetime.now()

    encoded_thought = tokenizer(thought, return_tensors='pt')

    output = model(**encoded_thought)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    for i in range(len(scores)):
    
        l = labels[i]
        s = scores[i]

    
    scores_list = [scores[i] for i in range(len(scores))]
    labels_list = [labels[i] for i in range(len(scores))]
    zipped = list(zip(scores_list,labels_list))

    thought_entries_df.loc[len(thought_entries_df.index)] = [thought,scores[0],scores[1],scores[2], max(zipped)[1]]#,timestamp.strftime("%B %d, %Y, %H:%Mh")]

    

In [132]:
thought_entries_df

,Thoughts,Sent_Negative,Sent_Neutral,Sent_Positive,Sentiment


In [133]:
replies_shuffled = replies_text.sample(frac=1)

In [134]:
user_1_thoughts = replies_shuffled.iloc[2000:4000]

user_1_thoughts

14869    I gained a pound after being on a diet for a week
3360     I forgot to order products for the store I wor...
1579     My brother and I have been really getting into...
3711               i would never try to wave to him again.
13262                                  I hurt other people
                               ...                        
11017                     My acquaintance doesn't like me.
8527     IT would just mean that I was annoyed.  I woul...
2186     I am useless and cannot come up with words to ...
9767     Something must be wrong with me to get rejecte...
561                                             I'm boring
Name: Reply, Length: 2000, dtype: object

In [135]:
# user_1_thoughts.to_csv('user_2_thoughts.csv', index = False)

In [136]:
user_1_thoughts

14869    I gained a pound after being on a diet for a week
3360     I forgot to order products for the store I wor...
1579     My brother and I have been really getting into...
3711               i would never try to wave to him again.
13262                                  I hurt other people
                               ...                        
11017                     My acquaintance doesn't like me.
8527     IT would just mean that I was annoyed.  I woul...
2186     I am useless and cannot come up with words to ...
9767     Something must be wrong with me to get rejecte...
561                                             I'm boring
Name: Reply, Length: 2000, dtype: object

In [139]:
thought_entry_specialist('hello im cool')

,Thoughts,Sent_Negative,Sent_Neutral,Sent_Positive,Sentiment
0,hello im cool,0.004106,0.08133,0.914565,Positive
